# Projeto Integrador 4

<img src="https://www.faesa.br/hubfs/logo-faesa-branco.png" style="background-color:DarkBlue;width:250px;float:left;"/>
<br><br><br><br><br>
<table style="float:left;">
    <tr>
        <td>
            <p><strong>FAESA CENTRO UNIVERSITÁRIO</strong></p>
        </td>
        <td>
            <p></p>
        </td>
        <td>
            <p><strong>Valor:</strong>10,0 (dez) pontos</p>
        </td>
    </tr>
    <tr>
        <td>
            <p><strong>Unidade: </strong>Unidade de Computação e Sistemas</p>
        </td>
        <td>
            <p><strong>Curso: </strong>Ciência da Computação e Sistemas de Informação</p>
        </td>
        <td>
            <p></p>
        </td>
    </tr>
    <tr>
        <td>
            <p><strong>Disciplina: </strong>Projeto Integrador Computação IV</p>
        </td>
        <td>
            <p><strong>Professor: </strong>Howard Roatti</p>
        </td>
        <td>
            <p><strong>Nota:</strong>_______________</p>
        </td>
    </tr>
    <tr>
        <td>
            <p><strong>Nome dos Integrantes: Gabriel Bremenkamp, Gabriel Tozato, Natan Campos</strong></p>
        </td>
</table>


# Exploração dos dados:

## Informações do dataset

## Bibliotecas

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns #Lib voltada para plot de graficos, baseada na lib matplotlib
import math

#Scikit Learn é uma lib open source de aprendizagem de maquina.
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler #Lib voltada a balanceamento de dados
from sklearn.model_selection import cross_val_score, RepeatedKFold, train_test_split, train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB



# Funções

In [2]:
def ler(a):
    resultado = []
    if a == 1:
        arq = open("tweets_valid.txt","r",encoding="utf8")
    else if a==0:
        arq = open("tweets.txt","r",encoding="utf8")
    linhas = arq.readlines()
    for linha in linhas:
        resultado.append(linha+'\t')
    return resultado

In [3]:
#TRANSFORMANDO O DATAFRAME EM UMA MATRIZ DE TERMOS
def matrixTweet(df0):
    x = np.array(df0['tweets'])
    result = []
    for i in x:
        i2 = i.replace(',','')
        i2 = i.replace('.','')
        i2 = i.replace(':','')
        i2 = i.replace('\n','')
        i2 = i.replace('\t','')
        i2 = i.replace('  ',' ')
        i2 = i.split()
        result.append(i2)
    return result

In [4]:
#metodo que realiza a quantidade de termos por classe
def contar_termos(termos, classe):
  quant_termos = pd.DataFrame(termos, columns=['Termo'])
  quant_termos['Quantidade'] = 0

  quant_termo = 0
  for termo in termos:
    for documento in classe:
      if termo in documento:
        quant_termo+=1
    quant_termos.loc[quant_termos['Termo']==termo, 'Quantidade'] = quant_termo
  return quant_termos

In [5]:
#calcular ICF
def icfCalc(termo, classe0,classe1,classe2):
    Ci = 0
    result = 0
    C = 3
    if termo in classe0:
        Ci += 1
    if termo in classe1:
        Ci += 1
    if termo in classe2:
        Ci += 1
    if Ci == 0:
        result = 0
    else:
        result = math.log(1+(C/Ci))
    return result

In [6]:
#a = quantidade de documentos que possuem o termo na classe
#b = quantidade de documentos que possuem o termo fora da classe
#função: log(2 + a / max(1,b))
def relevance_freq(a, b):
  return math.log(2 + (a / (max(1,b))))

In [7]:
def rf_por_termo(df, classe_0, classe_1, classe_2):
    
  classes = [classe_0, classe_1, classe_2]
  i = 0
  
  for classe in classes:
    classe_rf = 'Classe_' + str(i)

    classe_pertence = classes[0]
    classe_nao_pertence_1 = classes[1]
    classe_nao_pertence_2 = classes[2]

    for termo in df['Termo']:
      pertence = int(classe_pertence.loc[quant_termos['Termo']==termo, 'Quantidade'])
      nao_pertence = int(classe_nao_pertence_1.loc[quant_termos['Termo']==termo, 'Quantidade'])
      nao_pertence += int(classe_nao_pertence_2.loc[quant_termos['Termo']==termo, 'Quantidade'])

      rf = relevance_freq(pertence, nao_pertence)

      relevancy_frequency.loc[relevancy_frequency['Termo']==termo, classe_rf] = rf
    
    classe_atual = classes[0]
    classe_proxima = classes[1]
    classe_ultima = classes[2]
    classes[0] = classe_proxima
    classes[1] = classe_ultima
    classes[2] = classe_atual
    i += 1

In [8]:
def icfPorTermo(termos, classe_0, classe_1, classe_2,df):
    result = []
    vlr = 0
    
    
    
    for termo in termos:
        vlr = icfCalc(termo, classe_0,classe_1,classe_2)
        result.append(vlr)
        
    result = np.array(result)
    df['Classe_0'] = result
    df['Classe_1'] = result
    df['Classe_2'] = result
    
    return df

In [9]:
#GERA UM DATAFRAME NOVO USANDO O RESULTADO DO RELEVANCY FREQUENCI AGRUPANDO POR TWEET E USANDO OS TERMOS
#COMO COLUNAS

def vetorizer(dataframe,dataframePonderacao,relevancy_frequency,classe,classe2):
    dfResult = dataframePonderacao
    linhas = []
    header = []
    i=0
    for x in dataframe:
        linha = []
        for column in dfResult:
            if column in x:
                linha.append(float(relevancy_frequency[classe2].loc[relevancy_frequency['Termo'] == column]))
            else:
                linha.append(0)
        linhas.append(linha)
    df = pd.DataFrame(linhas,
                  columns = dfResult.columns.values.tolist())
    return df

In [10]:
def get_models():
	models = list()
	models.append(Lasso())
	models.append(Ridge())
	models.append(DecisionTreeRegressor())
	models.append(LinearRegression())
	return models

In [11]:
def evaluate_model(model):
	scores = cross_validate(model, df_train_encoded, df_train[outcome], scoring=scoring, cv=kfold, n_jobs=-1)
	return scores

##  Carregando os dados

In [17]:
#Carregando os tweets e convertendo eles para DataFrame
txt = ler(1)
txtAntigo = ler(0)
dfNovo = pd.DataFrame(txt, columns = ['tweets'])
dfNovo['texto'] = 1
dfAntigo = pd.DataFrame(txtAntigo, columns = ['tweets'])
dfAntigo['texto'] = 0
df = pd.concat([dfAntigo,dfNovo])
df_valid = df.drop_duplicates()
df_valid

,tweets,texto
0,"Eleições 2022: PDT anuncia apoio a Lula, e Cir...",0
1,eleições 2022 ou modern family?\n\t,0
2,O PDT anunciou apoio a Lula no segundo turno ...,0
3,Simone Tebet pretende encontrar Lula pessoalm...,0
4,A interpretação é de cada um. Eleições 2022\n\t,0
...,...,...
295,"Um partido, Um candidato não pode sequestrar a...",1
296,"— Jair Bolsonaro e o presidente do PL, Valdem...",1
297,Vazou vídeo do Bolsonaro assistindo o Bonner a...,1
298,"CALMA, XANDÃO! Tô só repassando a notícia da “...",1


# Classificação dos dados

In [26]:
#Transformando o dataframe em unidimensional para aplicar o vectorizer.
x = np.array(df_valid.drop(columns=['texto']))
vectorizer = CountVectorizer()
flat_x = x.flatten()
xVet = vectorizer.fit_transform(flat_x)

In [34]:
#Criando um dataframe e ordenando, utilizando as quantidades e os termos como colunas para poder fazer o plot
#Deixei os 10 termos que mais se repetem.

d = vectorizer.vocabulary_
d2 = pd.DataFrame(d.items(), columns=['Termo', 'Quantidade'])
df2 = d2.sort_values(by='Quantidade', ascending=False)
df2 = df2[df2.Quantidade < 5724]
df2
dfPonderacao = pd.DataFrame(df2['Termo'])
dfPonderacao = dfPonderacao.set_index('Termo')
dfPonderacao = dfPonderacao.T
df2

,Termo,Quantidade
1002,útil,5723
5195,únicos,5722
5120,único,5721
209,única,5720
3932,últimos,5719
...,...,...
2225,02,4
5509,01,3
2367,00hs,2
4796,000,1


In [35]:
#Inicializando a variavel com o Kmeans e treinando o modelo.

kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(xVet)

KMeans(n_clusters=3, random_state=0)

In [37]:
#Incluindo uma nova coluna no dataframe com o nome "Classification" que indica a que grupo pertence cada tweet, 0 1 ou 2

df['Classification'] = kmeans.labels_
df

,tweets,texto,Classification
0,"Eleições 2022: PDT anuncia apoio a Lula, e Cir...",0,0
1,eleições 2022 ou modern family?\n\t,0,0
2,O PDT anunciou apoio a Lula no segundo turno ...,0,0
3,Simone Tebet pretende encontrar Lula pessoalm...,0,1
4,A interpretação é de cada um. Eleições 2022\n\t,0,0
...,...,...,...
295,"Um partido, Um candidato não pode sequestrar a...",1,2
296,"— Jair Bolsonaro e o presidente do PL, Valdem...",1,1
297,Vazou vídeo do Bolsonaro assistindo o Bonner a...,1,1
298,"CALMA, XANDÃO! Tô só repassando a notícia da “...",1,0


In [42]:
#Daqui em diante eu vou plotar graficos correspondentes e termos que mais se repetem em cada grupo.
dff = df.drop(columns=['texto'])

df0 = dff.loc[df['Classification'] == 0]
df1 = dff.loc[df['Classification'] == 1]
df2 = dff.loc[df['Classification'] == 2]

df0 = df0.drop(columns=['Classification'])
df1 = df1.drop(columns=['Classification'])
df2 = df2.drop(columns=['Classification'])

x0 = np.array(df0)
x1 = np.array(df1)
x2 = np.array(df2)

In [44]:
#Grupo 1 - Nesse grupo se destacam os termos "Vitória" "Vencedor" "Vencer" portanto o classifiquei como Positivo

vectorizer = CountVectorizer()
flat_x0 = x0.flatten()
x0Vet = vectorizer.fit_transform(flat_x0)

d0 = vectorizer.vocabulary_
d02 = pd.DataFrame(d0.items(), columns=['Termo', 'Quantidade'])
df0 = d02.sort_values(by='Quantidade', ascending=False).head(50)
df0

,Termo,Quantidade
1601,وها,3663
1612,ونذكر,3662
1624,ودعم,3661
1606,وأعلن,3660
1620,منه,3659
1621,من,3658
1608,لولا,3657
1625,لحسم,3656
1618,كانت,3655
1615,قيادات,3654


In [45]:
#Grupo 2 - Nesse grupo a pesar de ter um alto numero de termos negativos ele ainda tem uma quantidade menor do que no segundo
#grupo, portanto o classifiquei como neutro, pois não tem mais positivos que o primeiro e nem tem mais negativos do que o segundo.

vectorizer = CountVectorizer()
flat_x1 = x1.flatten()
x1Vet = vectorizer.fit_transform(flat_x1)

d1 = vectorizer.vocabulary_
d12 = pd.DataFrame(d1.items(), columns=['Termo', 'Quantidade'])
df1 = d12.sort_values(by='Quantidade', ascending=False).head(50)

In [46]:
#Grupo 3 esse é o grupo ocm maior classificação, tem os maiores termos sendo negativos, "Odio" "Zombam" "Zombavam"

vectorizer = CountVectorizer()
flat_x2 = x2.flatten()
x2Vet = vectorizer.fit_transform(flat_x2)

d2 = vectorizer.vocabulary_
d22 = pd.DataFrame(d2.items(), columns=['Termo', 'Quantidade'])
df2 = d22.sort_values(by='Quantidade', ascending=False).head(50)
df2 = df2[df2.Quantidade < 2939]

# Ponderações

In [47]:
#Transformando o dataframe em unidimensional para aplicar o vectorizer.

flat_x = x.flatten()
xVet = vectorizer.fit_transform(flat_x)

#Imprimindo todos os termos encontrados no Dataframe.

termos = vectorizer.get_feature_names_out()

kmeans.fit(xVet)

df['Classification'] = kmeans.labels_

vectorizer = CountVectorizer()

x_0 = np.array(df['tweets'].loc[df['Classification'] == 0])
x_1 = np.array(df['tweets'].loc[df['Classification'] == 1])
x_2 = np.array(df['tweets'].loc[df['Classification'] == 2])

quant_termos = pd.DataFrame(termos, columns=['Termo'])
quant_termos['Quantidade'] = 0

In [48]:
classe_0 = contar_termos(termos, x_0)
classe_1 = contar_termos(termos, x_1)
classe_2 = contar_termos(termos, x_2)

In [49]:
relevancy_frequency = pd.DataFrame(termos, columns=['Termo'])
relevancy_frequency['Classe_0'] = 0
relevancy_frequency['Classe_1'] = 0
relevancy_frequency['Classe_2'] = 0

In [50]:
dfIcf = pd.DataFrame(termos, columns=['Termo'])

In [51]:
rf_por_termo(relevancy_frequency, classe_0, classe_1, classe_2)

In [56]:
dfNovo = df.loc[df['texto'] == 1]
dfAntigo = df.loc[df['texto'] == 0]

In [53]:
class0 = np.array(d02['Termo'])
class1 = np.array(d12['Termo'])
class2 = np.array(d22['Termo'])

icf = icfPorTermo(dfIcf['Termo'],class0,class1,class2,dfIcf)

In [63]:
#SEPARANDO AS MATRIZES POR GRUPO "CLASSIFICATION"
#novo
dfNovo0 = dfNovo.loc[dfNovo['Classification'] == 0]
dfNovo0 = matrixTweet(dfNovo0)
dfNovo1 = dfNovo.loc[dfNovo['Classification'] == 1]
dfNovo1 = matrixTweet(dfNovo1)
dfNovo2 = dfNovo.loc[dfNovo['Classification'] == 2]
dfNovo2 = matrixTweet(dfNovo2)
dfAntigo0 = dfAntigo.loc[dfAntigo['Classification'] == 0]
dfAntigo0 = matrixTweet(dfAntigo0)
dfAntigo1 = dfAntigo.loc[dfAntigo['Classification'] == 1]
dfAntigo1 = matrixTweet(dfAntigo1)
dfAntigo2 = dfAntigo.loc[dfAntigo['Classification'] == 2]
dfAntigo2 = matrixTweet(dfAntigo2)

In [64]:
dfRF = relevancy_frequency[['Classe_0','Classe_1','Classe_2']]
dfICF = dfIcf[['Classe_0','Classe_1','Classe_2']]
icfBased = dfICF.mul(dfRF, fill_value = 0)
icfBased['Termo'] = dfIcf['Termo']

In [65]:
#antigo
dfPonderacaoRF0_antigo = vetorizer(dfAntigo0,dfPonderacao,relevancy_frequency,'Positivo','Classe_0')
dfPonderacaoRF1_antigo = vetorizer(dfAntigo1,dfPonderacao,relevancy_frequency,'Neutro','Classe_1')
dfPonderacaoRF2_antigo = vetorizer(dfAntigo2,dfPonderacao,relevancy_frequency,'Negativo','Classe_2')
dfPonderacaoICF0_antigo = vetorizer(dfAntigo0,dfPonderacao,dfIcf,'Positivo','Classe_0')
dfPonderacaoICF1_antigo = vetorizer(dfAntigo1,dfPonderacao,dfIcf,'Neutro','Classe_1')
dfPonderacaoICF2_antigo = vetorizer(dfAntigo2,dfPonderacao,dfIcf,'Negativo','Classe_2')
dfPonderacaoICFBASED0_antigo = vetorizer(dfAntigo0,dfPonderacao,icfBased,'Positivo','Classe_0')
dfPonderacaoICFBASED1_antigo = vetorizer(dfAntigo1,dfPonderacao,icfBased,'Neutro','Classe_1')
dfPonderacaoICFBASED2_antigo = vetorizer(dfAntigo2,dfPonderacao,icfBased,'Negativo','Classe_2')
#novo
dfPonderacaoRF0_novo = vetorizer(dfNovo0,dfPonderacao,relevancy_frequency,'Positivo','Classe_0')
dfPonderacaoRF1_novo = vetorizer(dfNovo1,dfPonderacao,relevancy_frequency,'Neutro','Classe_1')
dfPonderacaoRF2_novo = vetorizer(dfNovo2,dfPonderacao,relevancy_frequency,'Negativo','Classe_2')
dfPonderacaoICF0_novo = vetorizer(dfNovo0,dfPonderacao,dfIcf,'Positivo','Classe_0')
dfPonderacaoICF1_novo = vetorizer(dfNovo1,dfPonderacao,dfIcf,'Neutro','Classe_1')
dfPonderacaoICF2_novo = vetorizer(dfNovo2,dfPonderacao,dfIcf,'Negativo','Classe_2')
dfPonderacaoICFBASED0_novo = vetorizer(dfNovo0,dfPonderacao,icfBased,'Positivo','Classe_0')
dfPonderacaoICFBASED1_novo = vetorizer(dfNovo1,dfPonderacao,icfBased,'Neutro','Classe_1')
dfPonderacaoICFBASED2_novo = vetorizer(dfNovo2,dfPonderacao,icfBased,'Negativo','Classe_2')



In [66]:
#Antigo
dfPonderacaoRF0_antigo['Classification'] = 'Positivo'
dfPonderacaoRF1_antigo['Classification'] = 'Neutro'
dfPonderacaoRF2_antigo['Classification'] = 'Negativo'
dfPonderacaoICF0_antigo['Classification'] = 'Positivo'
dfPonderacaoICF1_antigo['Classification'] = 'Neutro'
dfPonderacaoICF2_antigo['Classification'] = 'Negativo'
dfPonderacaoICFBASED0_antigo['Classification'] = 'Positivo'
dfPonderacaoICFBASED1_antigo['Classification'] = 'Neutro'
dfPonderacaoICFBASED2_antigo['Classification'] = 'Negativo'
#Novo
dfPonderacaoRF0_novo['Classification'] = 'Positivo'
dfPonderacaoRF1_novo['Classification'] = 'Neutro'
dfPonderacaoRF2_novo['Classification'] = 'Negativo'
dfPonderacaoICF0_novo['Classification'] = 'Positivo'
dfPonderacaoICF1_novo['Classification'] = 'Neutro'
dfPonderacaoICF2_novo['Classification'] = 'Negativo'
dfPonderacaoICFBASED0_novo['Classification'] = 'Positivo'
dfPonderacaoICFBASED1_novo['Classification'] = 'Neutro'
dfPonderacaoICFBASED2_novo['Classification'] = 'Negativo'

In [67]:
#JUNTANDO TODOS OS DATAFRAMES
#Antigo
df_valid_RF_antigo = pd.concat([dfPonderacaoRF0_antigo, dfPonderacaoRF1_antigo,dfPonderacaoRF2_antigo])
df_valid_ICF_antigo = pd.concat([dfPonderacaoICF0_antigo, dfPonderacaoICF1_antigo,dfPonderacaoICF2_antigo])
df_valid_ICFBASED_antigo = pd.concat([dfPonderacaoICFBASED0_antigo, dfPonderacaoICFBASED1_antigo,dfPonderacaoICFBASED2_antigo])
#Novo
df_valid_RF_novo = pd.concat([dfPonderacaoRF0_novo, dfPonderacaoRF1_novo,dfPonderacaoRF2_novo])
df_valid_ICF_novo = pd.concat([dfPonderacaoICF0_novo, dfPonderacaoICF1_novo,dfPonderacaoICF2_novo])
df_valid_ICFBASED_novo = pd.concat([dfPonderacaoICFBASED0_novo, dfPonderacaoICFBASED1_novo,dfPonderacaoICFBASED2_novo])

# Treinando os modelos

## Relevancy frequency

### KNN - RF

In [70]:
#Definindo uma baseline utilizando KNN
baselineKNN_RF = KNeighborsClassifier(n_neighbors=20)

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_RF_antigo.drop(columns=['Classification'])
y = df_valid_RF_antigo['Classification']

#Dividindo em treinamento e teste
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=1)

#treinando o modelo
baselineKNN_RF.fit(X_train, y_train)

#Imprimindo o score
predKNN = baselineKNN_RF.predict(X_valid)
accuracy_score(y_valid, predKNN)

0.7433333333333333

In [71]:
#Pontuação
classification = classification_report(y_valid, predKNN)
print(classification)

              precision    recall  f1-score   support

    Negativo       0.00      0.00      0.00        42
      Neutro       0.00      0.00      0.00        35
    Positivo       0.74      1.00      0.85       223

    accuracy                           0.74       300
   macro avg       0.25      0.33      0.28       300
weighted avg       0.55      0.74      0.63       300



C:\Users\gabriel.sebastiao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gabriel.sebastiao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gabriel.sebastiao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [73]:
#Validação

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_RF_novo.drop(columns=['Classification'])
y = df_valid_RF_novo['Classification']

#Imprimindo o score
predKNN = baselineKNN_RF.predict(X)
accuracy_score(y, predKNN)

0.49333333333333335

### Naive Bayes - RF

In [74]:
#Definindo uma baseline utilizando KNN
naiveModel_RF = GaussianNB()

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_RF_antigo.drop(columns=['Classification'])
y = df_valid_RF_antigo['Classification']

#Dividindo em treinamento e teste
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=1)

#treinando o modelo
naiveModel_RF.fit(X_train, y_train)

#Imprimindo o score
predNaive = naiveModel_RF.predict(X_valid)
accuracy_score(y_valid, predNaive)

0.4066666666666667

In [75]:
#Pontuação
classification = classification_report(y_valid, predNaive)
print(classification)

              precision    recall  f1-score   support

    Negativo       0.14      0.45      0.21        42
      Neutro       0.31      0.29      0.30        35
    Positivo       0.71      0.42      0.53       223

    accuracy                           0.41       300
   macro avg       0.39      0.39      0.35       300
weighted avg       0.58      0.41      0.46       300



In [76]:
#Validação

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_RF_novo.drop(columns=['Classification'])
y = df_valid_RF_novo['Classification']

#Imprimindo o score
predNaive = naiveModel_RF.predict(X)
accuracy_score(y, predNaive)

0.3433333333333333

## ICF

### KNN - ICF

In [77]:
#Definindo uma baseline utilizando KNN
baselineKNN_ICF = KNeighborsClassifier(n_neighbors=20)

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_ICF_antigo.drop(columns=['Classification'])
y = df_valid_ICF_antigo['Classification']

#Dividindo em treinamento e teste
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=1)

#treinando o modelo
baselineKNN_ICF.fit(X_train, y_train)

#Imprimindo o score
predKNN = baselineKNN_ICF.predict(X_valid)
accuracy_score(y_valid, predKNN)

0.7433333333333333

In [78]:
#Pontuação
classification = classification_report(y_valid, predKNN)
print(classification)

              precision    recall  f1-score   support

    Negativo       0.00      0.00      0.00        42
      Neutro       0.00      0.00      0.00        35
    Positivo       0.74      1.00      0.85       223

    accuracy                           0.74       300
   macro avg       0.25      0.33      0.28       300
weighted avg       0.55      0.74      0.63       300



C:\Users\gabriel.sebastiao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gabriel.sebastiao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gabriel.sebastiao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [80]:
#Validação

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_ICF_novo.drop(columns=['Classification'])
y = df_valid_ICF_novo['Classification']

#Imprimindo o score
predKNN = baselineKNN_ICF.predict(X)
accuracy_score(y, predKNN)

0.4866666666666667

### Naive Bayes - ICF

In [81]:
#Definindo uma baseline utilizando KNN
naiveModel_ICF = GaussianNB()

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_ICF_antigo.drop(columns=['Classification'])
y = df_valid_ICF_antigo['Classification']

#Dividindo em treinamento e teste
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=1)

#treinando o modelo
naiveModel_ICF.fit(X_train, y_train)

#Imprimindo o score
predNaive = naiveModel_ICF.predict(X_valid)
accuracy_score(y_valid, predNaive)

0.44333333333333336

In [82]:
#Pontuação
classification = classification_report(y_valid, predNaive)
print(classification)

              precision    recall  f1-score   support

    Negativo       0.20      0.64      0.31        42
      Neutro       0.28      0.43      0.34        35
    Positivo       0.80      0.41      0.54       223

    accuracy                           0.44       300
   macro avg       0.43      0.49      0.40       300
weighted avg       0.65      0.44      0.48       300



In [83]:
#Validação

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_ICF_novo.drop(columns=['Classification'])
y = df_valid_ICF_novo['Classification']

#Imprimindo o score
predNaive = naiveModel_ICF.predict(X)
accuracy_score(y, predNaive)

0.37666666666666665

## ICF-BASED

### KNN - ICF-BASED

In [84]:
#Definindo uma baseline utilizando KNN
baselineKNN_ICFBASED = KNeighborsClassifier(n_neighbors=20)

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_ICFBASED_antigo.drop(columns=['Classification'])
y = df_valid_ICFBASED_antigo['Classification']

#Dividindo em treinamento e teste
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=1)

#treinando o modelo
baselineKNN_ICFBASED.fit(X_train, y_train)

#Imprimindo o score
predKNN = baselineKNN_ICFBASED.predict(X_valid)
accuracy_score(y_valid, predKNN)

0.7433333333333333

In [85]:
#Pontuação
classification = classification_report(y_valid, predKNN)
print(classification)

              precision    recall  f1-score   support

    Negativo       0.00      0.00      0.00        42
      Neutro       0.00      0.00      0.00        35
    Positivo       0.74      1.00      0.85       223

    accuracy                           0.74       300
   macro avg       0.25      0.33      0.28       300
weighted avg       0.55      0.74      0.63       300



C:\Users\gabriel.sebastiao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gabriel.sebastiao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gabriel.sebastiao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [86]:
#Validação

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_ICFBASED_novo.drop(columns=['Classification'])
y = df_valid_ICFBASED_novo['Classification']

#Imprimindo o score
predKNN = baselineKNN_ICF.predict(X)
accuracy_score(y, predKNN)

0.4866666666666667

### Naive Bayes - ICF-BASED

In [87]:
#Definindo uma baseline utilizando KNN
naiveModel_ICFBASED = GaussianNB()

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_ICFBASED_antigo.drop(columns=['Classification'])
y = df_valid_ICFBASED_antigo['Classification']

#Dividindo em treinamento e teste
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=1)

#treinando o modelo
naiveModel_ICFBASED.fit(X_train, y_train)

#Imprimindo o score
predNaive = naiveModel_ICFBASED.predict(X_valid)
accuracy_score(y_valid, predNaive)

0.45666666666666667

In [88]:
#Pontuação
classification = classification_report(y_valid, predNaive)
print(classification)

              precision    recall  f1-score   support

    Negativo       0.17      0.55      0.26        42
      Neutro       0.38      0.40      0.39        35
    Positivo       0.77      0.45      0.57       223

    accuracy                           0.46       300
   macro avg       0.44      0.47      0.41       300
weighted avg       0.64      0.46      0.50       300



In [89]:
#Validação

#Recebendo o dataset segundo o filtro do random forest
X = df_valid_ICF_novo.drop(columns=['Classification'])
y = df_valid_ICF_novo['Classification']

#Imprimindo o score
predNaive = naiveModel_ICF.predict(X)
accuracy_score(y, predNaive)

0.37666666666666665

Após treinar os modelos, percebi que a maior pontuação vem do algoritmo de vetorização ICF junto com o modelo KNN.